In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Read the data and observe the distribution of missing values. The raw data has 28800 rows with many missing values. However, it seems that each row has at most ten missing value. This suggests the possibility of building a predictive model to impute the missing values. 

In [70]:
df = pd.read_csv('../data/train.csv')

print(f"{df.shape}\n")

df = df.dropna(thresh=50)

print(f"{df.shape}\n")


(28800, 60)

(22568, 60)



In [60]:
df.tail()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
28758,28758,Intermediate,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,23.895
28764,28764,High,No,Poor,Yes,2.0,8.0,"TBI +- Other, <=cGy",No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,3.857
28770,28770,Intermediate,Yes,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,MEL,8.0,No,2.0,No,10.0,1.0,7.206
28780,28780,Intermediate,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,1.0,5.932
28788,28788,Intermediate,Yes,Other,No,2.0,8.0,No TBI,No,6.0,...,HvG non-permissive,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,24.115


In the following, we demonstrate one strategy to impute the data with a predictive model, using the column "dri_score" as an example. 
1. Seperate those rows with missing value in "dri_score" and those without missing values in "dri_score".
2. Get dummies for the categorical variables, and include NA as a category for each variables
3. Impute the numerical variables with a simple imputer
4. Train a logistic regression model using those rows without missing values in "dri_score" and predict.

In [83]:
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

df_na = df.loc[df["dri_score"].isna()].copy()

df_na_y = df_na["dri_score"]

df_na_X = df_na.drop(['ID', 'dri_score'], axis=1)

df_na_X = pd.get_dummies(df_na_X, dummy_na=True)

for key in df_na_X.keys():
    if df_na_X.dtypes[key] == float:
        imp = SimpleImputer(strategy="mean")
        df_na_X[[key]] = imp.fit_transform(df_na_X[[key]])

df_nonna = df.dropna(subset=["dri_score"]).copy()

df_nonna_y = df_nonna["dri_score"]

df_nonna_X = df_nonna.drop(['ID', 'dri_score'], axis=1)

df_nonna_X = pd.get_dummies(df_nonna_X, dummy_na=True)

for key in df_nonna_X.keys():
    if df_nonna_X.dtypes[key] == float:
        imp = SimpleImputer(strategy="mean")
        df_nonna_X[[key]] = imp.fit_transform(df_nonna_X[[key]])

regressor = LogisticRegression()

regressor.fit(df_nonna_X.loc[:, df_na_X.keys()], df_nonna_y)

regressor.predict(df_na_X.loc[:, df_na_X.keys()])
      


/opt/homebrew/Caskroom/miniforge/base/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['Intermediate', 'Intermediate', 'N/A - pediatric', 'Intermediate',
       'Intermediate', 'Intermediate', 'Intermediate', 'Intermediate',
       'Intermediate', 'Intermediate', 'N/A - pediatric', 'Intermediate',
       'Intermediate', 'N/A - pediatric', 'Intermediate',
       'N/A - pediatric', 'N/A - pediatric', 'Intermediate',
       'Intermediate', 'N/A - pediatric', 'Intermediate',
       'N/A - pediatric', 'N/A - pediatric', 'Intermediate',
       'Intermediate', 'Intermediate', 'Intermediate', 'Intermediate',
       'Intermediate', 'Intermediate', 'N/A - pediatric', 'Intermediate',
       'Intermediate', 'Intermediate', 'N/A - pediatric',
       'N/A - pediatric', 'Intermediate', 'N/A - pediatric',
       'Intermediate', 'Intermediate', 'N/A - pediatric', 'Intermediate',
       'Intermediate', 'N/A - pediatric', 'N/A - pediatric',
       'Intermediate', 'Intermediate', 'Intermediate', 'Intermediate',
       'N/A - pediatric', 'Intermediate', 'Intermediate', 'Intermediate',